In [1]:
import mnist_loader
training_data, validation_data, test_data, train_for_error_data = mnist_loader.load_data_wrapper()

In [3]:
"""
network.py
~~~~~~~~~~

A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None, test_data2=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)

            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)
            
            if test_data2:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data2), len(test_data2))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))


In [4]:
net = Network([784, 30, 10])

In [5]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data, test_data2=train_for_error_data)

Epoch 0: 8889 / 10000
Epoch 0: 44352 / 50000
Epoch 1: 9177 / 10000
Epoch 1: 46028 / 50000
Epoch 2: 9283 / 10000
Epoch 2: 46542 / 50000
Epoch 3: 9310 / 10000
Epoch 3: 46891 / 50000
Epoch 4: 9370 / 10000
Epoch 4: 47122 / 50000
Epoch 5: 9392 / 10000
Epoch 5: 47399 / 50000
Epoch 6: 9428 / 10000
Epoch 6: 47501 / 50000
Epoch 7: 9410 / 10000
Epoch 7: 47554 / 50000
Epoch 8: 9416 / 10000
Epoch 8: 47657 / 50000
Epoch 9: 9422 / 10000
Epoch 9: 47747 / 50000
Epoch 10: 9458 / 10000
Epoch 10: 47864 / 50000
Epoch 11: 9422 / 10000
Epoch 11: 47819 / 50000
Epoch 12: 9445 / 10000
Epoch 12: 47972 / 50000
Epoch 13: 9453 / 10000
Epoch 13: 48005 / 50000
Epoch 14: 9482 / 10000
Epoch 14: 48135 / 50000
Epoch 15: 9454 / 10000
Epoch 15: 48150 / 50000
Epoch 16: 9477 / 10000
Epoch 16: 48199 / 50000
Epoch 17: 9493 / 10000
Epoch 17: 48365 / 50000
Epoch 18: 9491 / 10000
Epoch 18: 48335 / 50000
Epoch 19: 9458 / 10000
Epoch 19: 48064 / 50000
Epoch 20: 9467 / 10000
Epoch 20: 48264 / 50000
Epoch 21: 9514 / 10000
Epoch 21: 

### 2 Hidden Layers

In [ ]:
net = Network([784, 30, 30, 10])

In [7]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data, test_data2=train_for_error_data)

Epoch 0: 9100 / 10000
Epoch 0: 45509 / 50000
Epoch 1: 9203 / 10000
Epoch 1: 46278 / 50000
Epoch 2: 9305 / 10000
Epoch 2: 46860 / 50000
Epoch 3: 9312 / 10000
Epoch 3: 46856 / 50000
Epoch 4: 9361 / 10000
Epoch 4: 47182 / 50000
Epoch 5: 9448 / 10000
Epoch 5: 47670 / 50000
Epoch 6: 9396 / 10000
Epoch 6: 47436 / 50000
Epoch 7: 9473 / 10000
Epoch 7: 47816 / 50000
Epoch 8: 9504 / 10000
Epoch 8: 47878 / 50000
Epoch 9: 9486 / 10000
Epoch 9: 47957 / 50000
Epoch 10: 9477 / 10000
Epoch 10: 48051 / 50000
Epoch 11: 9496 / 10000
Epoch 11: 48093 / 50000
Epoch 12: 9467 / 10000
Epoch 12: 47967 / 50000
Epoch 13: 9528 / 10000
Epoch 13: 48299 / 50000
Epoch 14: 9495 / 10000
Epoch 14: 48194 / 50000
Epoch 15: 9500 / 10000
Epoch 15: 48336 / 50000
Epoch 16: 9504 / 10000
Epoch 16: 48370 / 50000
Epoch 17: 9542 / 10000
Epoch 17: 48491 / 50000
Epoch 18: 9532 / 10000
Epoch 18: 48530 / 50000
Epoch 19: 9514 / 10000
Epoch 19: 48491 / 50000
Epoch 20: 9497 / 10000
Epoch 20: 48436 / 50000
Epoch 21: 9521 / 10000
Epoch 21: 